# IDEALISTA DATA BASE

In [2]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import os
import unidecode

### READ AND CLEAN DATASET

In [4]:
df = pd.read_csv(r"C:\Users\faval\Desktop\Ironhack\DataAnalytics\final_project\data\raw\idealista_output_etl.csv", delimiter=';')

C:\Users\faval\AppData\Local\Temp\ipykernel_17152\2007474822.py:1: DtypeWarning: Columns (24,25,61,80,84) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\faval\Desktop\Ironhack\DataAnalytics\final_project\data\raw\idealista_output_etl.csv", delimiter=';')


In [40]:
df.shape

(379204, 25)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# SELECT THE SEGOVIA PROVINCE TO IMPORT A LIGHTER EXCEL FILE FOR BETTER UNDERSTANDING OF THE COLUMNS AND DATA WITHIN IT
segovia_df = df[df['ad_province'] == 'Segovia']

In [ ]:
save_path = r'C:\Users\faval\Desktop\Ironhack\DataAnalytics\final_project\data\clean'
file_name = 'segovia_df.csv'
full_path = os.path.join(save_path, file_name)
segovia_df.to_csv(full_path, index=False, encoding='utf-8')

In [8]:
#SELECT COLUMNS TO KEEP
columns_to_keep = ["ad_province", "ad_town", "ad_postalcode", "ad_operation", "ad_typology", 
    "ad_propertytype", "ad_hasgarden", "ad_hasparkingspace", 
    "ad_hasswimmingpool", "ad_area", "ad_price", "ad_roomnumber", 
    "ad_activationdate", "ad_bathnumber", "ad_constructionyear", "ad_floornumber", 
    "ad_haslift", "ad_hasterrace", "ad_isintopfloor", "ad_latitude", "ad_longitude", 
    "adstats_contactsbyemail", "adstats_savedasfavorite", "adstats_sendtofriend", 
    "adstats_visits"]
df = df[columns_to_keep]

In [9]:
#FOR THIS STUDY ONLY KEEP PENINSULAR SPAIN
provinces_to_exclude = ["Ceuta", "País Vasco Francés", "Melilla", "Cerdanya Francesa","Las Palmas","Santa Cruz de Tenerife", "Balears (Illes)", "Andorra" ]
df = df[~df['ad_province'].isin(provinces_to_exclude)]

In [11]:
df= df[df['ad_operation'] == 'Venta']

In [14]:
df= df[df['ad_typology'] == 'Piso']

In [16]:
# ad_floornumber: Define the replacements in a dictionary
replacements = {'st': 0,'bj': 0, 'nan': 0,'ss': 0,'-2': 0,'-1': 0,'BJ': 0,'en': 1}
df['ad_floornumber'] = df['ad_floornumber'].replace(replacements).fillna(0)

In [18]:
# HANDLE NULL VALUES: ad_haslift, ad_hasterrace, ad_isintopfloor
columns_to_fill = ['ad_haslift', 'ad_hasterrace', 'ad_isintopfloor']
# Fill NaN values with 0 for the specified columns
df[columns_to_fill] = df[columns_to_fill].fillna(0)

In [20]:
# CAST COLUMNS AS BOOLEAN: rename column, replace "Second Hand" and "New Construction" for 0 and 1
df = df.rename(columns={'ad_propertytype': 'ad_newconstruction'})
df['ad_newconstruction'] = df['ad_newconstruction'].replace({'Obra Nueva': 1, 'Segunda Mano': 0})

C:\Users\faval\AppData\Local\Temp\ipykernel_17152\3741720412.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ad_newconstruction'] = df['ad_newconstruction'].replace({'Obra Nueva': 1, 'Segunda Mano': 0})


In [22]:
# CAST COLUMNS AS BOOLEAN: first replace "No" and "Yes" for 0 and 1
columns_to_replace = ['ad_hasgarden', 'ad_hasparkingspace', 'ad_hasswimmingpool']
df[columns_to_replace] = df[columns_to_replace].replace({'Yes': 1, 'No': 0})

C:\Users\faval\AppData\Local\Temp\ipykernel_17152\1185828415.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[columns_to_replace] = df[columns_to_replace].replace({'Yes': 1, 'No': 0})


In [24]:
# CAST COLUMNS AS BOOLEAN
columns_to_cast = ['ad_haslift', 'ad_hasterrace', 'ad_isintopfloor', 
                   'ad_hasgarden', 'ad_hasparkingspace', 'ad_hasswimmingpool','ad_newconstruction']
df[columns_to_cast] = df[columns_to_cast].astype(bool)

In [26]:
# CAST COLUMNS AS INTEGER
columns_to_cast_as_int = ['ad_roomnumber', 'ad_bathnumber', 'ad_floornumber']
df[columns_to_cast_as_int] = df[columns_to_cast_as_int].astype(int)

In [28]:
# CAST COLUMNS AS DATE
df['ad_activationdate'] = pd.to_datetime(df['ad_activationdate'], format='%Y-%m-%d')

In [30]:
# CHECK FOR DUPLICATES
duplicates_exist = df.duplicated().any()
duplicates_exist

True

In [32]:
# DROP DUPLICATES
df = df.drop_duplicates()

In [34]:
# DROP NULL VALUES
df = df.dropna(subset=['adstats_contactsbyemail'])

In [36]:
# FILTER FOR LAST 5 YEARS
start_date = pd.Timestamp("2019-01-01")
df = df[(df['ad_activationdate'] >= start_date)]

In [38]:
# CONVERT CHARACTERS WITH ACCENTS
df['ad_province'] = df['ad_province'].apply(unidecode.unidecode)

### SAVE CLEAN DATASET

In [ ]:
save_path = r'C:\Users\faval\Desktop\Ironhack\DataAnalytics\final_project\data\clean'
file_name = 'clean_idealista_output_etl.csv'
full_path = os.path.join(save_path, file_name)
df.to_csv(full_path, index=False, encoding='utf-8')